In [ ]:
library(tidyverse)
library(data.table)
library(dplyr)
library(stringr)

In [ ]:
twitter_csv <- fread("UBC-ISD.csv", encoding="UTF-8")

In [ ]:

twitter_data <- twitter_csv |>
select(starts_with("twitter"))

colnames(twitter_data) <- str_replace(colnames(twitter_data), "twitter.tweet/", "")
colnames(twitter_data) <- str_replace(colnames(twitter_data), "twitter.user/", "")


colnames(twitter_data)

In [81]:
twitter_info<- twitter_data |>
select("originalText", "text", "possiblySensitive", "screenName", "created")

In [ ]:
library(stringr)

extract_hashtags <- function(x) {
  hashtags <- str_extract_all(x, "#\\S+")
  return(unlist(hashtags))
}

In [ ]:

twitter_info <- twitter_info |>
  mutate(hashtags = lapply(text, extract_hashtags))|>
  unnest(hashtags)|>
  mutate(hashtags = str_replace_all(hashtags, "\\.$", ""))|>
  filter(hashtags != "character(0)")

In [ ]:
twitter_info

In [ ]:
twitter_info_gg <- twitter_info |>
select(created, hashtags)|>
group_by(created, hashtags)|>
summarise(freq = n())|>
ungroup()

In [ ]:
twitter_info_graph <- twitter_info_gg|>
filter(hashtags %in% c("#COVID19","#COVID19ON","#vhc0N","#VHCDose2","#COVID19Vaccine"))
twitter_info_graph

In [ ]:
library(lubridate)

twitter_info_idk$created_rounded <- twitter_info_graph |>
 mutate(created_rounded = round_date(created, unit = "week"))


In [ ]:
twitter_info_idk

In [79]:
graph <- twitter_info_idk|>
ggplot(aes(x=created, y=freq, color=hashtags))+geom_point(alpha=0.1)

graph

ERROR: [1m[33mError[39m in `fortify()`:[22m
[1m[22m[33m![39m `data` must be a [34m<data.frame>[39m, or an object coercible by `fortify()`,
  not a list.


In [ ]:
twitter_info <- twitter_info |>
unnest(hashtags)

In [80]:
twitter_info

originalText,text,possiblySensitive,screenName,created,hashtags
<chr>,<chr>,<lgl>,<chr>,<dttm>,<chr>
,@PattyHajdu But astra zenica is a one dose vaccine! #cdnpoli,FALSE,GSawision,2021-05-04 11:32:15,#cdnpoli
RT @DrVivianS: Ongoing staffing crises continue to limit family access in #LTC. This is simply wrong @RodPhillips01. Amend the guidelines.…,Ongoing staffing crises continue to limit family access in #LTC. This is simply wrong @RodPhillips01. Amend the guidelines. Grant all fully vaccinated family members indoor & outdoor access to residents. No scheduling. No supervision. Only masking necessary at this point.,FALSE,KerryMcManus,2021-06-27 11:56:14,#LTC
RT @VaxHuntersCan: [ON] <U+2728><U+2728>DOWNSVIEW ARENA TODAY<U+2728><U+2728> <U+0001F4CD>1633 Wilson Ave <U+0001F5D3><U+FE0F>June 27 (10am-6pm) <U+2611><U+FE0F>1st dose 12+ all M postals <U+2611><U+FE0F>2nd dose 12+ live/wo…,[ON] <U+2728><U+2728>DOWNSVIEW ARENA TODAY<U+2728><U+2728> <U+0001F4CD>1633 Wilson Ave <U+0001F5D3><U+FE0F>June 27 (10am-6pm) <U+2611><U+FE0F>1st dose 12+ all M postals <U+2611><U+FE0F>2nd dose 12+ live/work/school in PRIORITY POSTALS Check poster carefully for details! https://t.co/NPDVg3PI07 #VHCDose2 #COVID19ON #vhcON,FALSE,thepuppydog26,2021-06-27 11:56:17,#VHCDose2
RT @VaxHuntersCan: [ON] <U+2728><U+2728>DOWNSVIEW ARENA TODAY<U+2728><U+2728> <U+0001F4CD>1633 Wilson Ave <U+0001F5D3><U+FE0F>June 27 (10am-6pm) <U+2611><U+FE0F>1st dose 12+ all M postals <U+2611><U+FE0F>2nd dose 12+ live/wo…,[ON] <U+2728><U+2728>DOWNSVIEW ARENA TODAY<U+2728><U+2728> <U+0001F4CD>1633 Wilson Ave <U+0001F5D3><U+FE0F>June 27 (10am-6pm) <U+2611><U+FE0F>1st dose 12+ all M postals <U+2611><U+FE0F>2nd dose 12+ live/work/school in PRIORITY POSTALS Check poster carefully for details! https://t.co/NPDVg3PI07 #VHCDose2 #COVID19ON #vhcON,FALSE,thepuppydog26,2021-06-27 11:56:17,#COVID19ON
RT @VaxHuntersCan: [ON] <U+2728><U+2728>DOWNSVIEW ARENA TODAY<U+2728><U+2728> <U+0001F4CD>1633 Wilson Ave <U+0001F5D3><U+FE0F>June 27 (10am-6pm) <U+2611><U+FE0F>1st dose 12+ all M postals <U+2611><U+FE0F>2nd dose 12+ live/wo…,[ON] <U+2728><U+2728>DOWNSVIEW ARENA TODAY<U+2728><U+2728> <U+0001F4CD>1633 Wilson Ave <U+0001F5D3><U+FE0F>June 27 (10am-6pm) <U+2611><U+FE0F>1st dose 12+ all M postals <U+2611><U+FE0F>2nd dose 12+ live/work/school in PRIORITY POSTALS Check poster carefully for details! https://t.co/NPDVg3PI07 #VHCDose2 #COVID19ON #vhcON,FALSE,thepuppydog26,2021-06-27 11:56:17,#vhcON
RT @VaxHuntersCan: [ON] <U+0001F489> OTTAWA POPUP <U+2B50><U+FE0F> OPH priority neighbourhoods <U+0001F4CD> @hudsonsbay Rideau Centre 73 Rideau St <U+0001F5D3><U+FE0F> July 3 9:45-4:30pm <U+0001F947>1st…,[ON] <U+0001F489> OTTAWA POPUP <U+2B50><U+FE0F> OPH priority neighbourhoods <U+0001F4CD> @hudsonsbay Rideau Centre 73 Rideau St <U+0001F5D3><U+FE0F> July 3 9:45-4:30pm <U+0001F947>1st DOSE 12+<U+0001F948>2nd DOSE 18+ & 28 days post mRNA/8 weeks AZ <U+2728>Bonus 20% off! NO OHIP Required Verify -> https://t.co/5gSHxBUpyt #VHCDose2 #COVID19ON #vhcON https://t.co/XLfFi9eTDq,FALSE,sexyfoodbyme,2021-07-03 10:07:36,#VHCDose2
RT @VaxHuntersCan: [ON] <U+0001F489> OTTAWA POPUP <U+2B50><U+FE0F> OPH priority neighbourhoods <U+0001F4CD> @hudsonsbay Rideau Centre 73 Rideau St <U+0001F5D3><U+FE0F> July 3 9:45-4:30pm <U+0001F947>1st…,[ON] <U+0001F489> OTTAWA POPUP <U+2B50><U+FE0F> OPH priority neighbourhoods <U+0001F4CD> @hudsonsbay Rideau Centre 73 Rideau St <U+0001F5D3><U+FE0F> July 3 9:45-4:30pm <U+0001F947>1st DOSE 12+<U+0001F948>2nd DOSE 18+ & 28 days post mRNA/8 weeks AZ <U+2728>Bonus 20% off! NO OHIP Required Verify -> https://t.co/5gSHxBUpyt #VHCDose2 #COVID19ON #vhcON https://t.co/XLfFi9eTDq,FALSE,sexyfoodbyme,2021-07-03 10:07:36,#COVID19ON
RT @VaxHuntersCan: [ON] <U+0001F489> OTTAWA POPUP <U+2B50><U+FE0F> OPH priority neighbourhoods <U+0001F4CD> @hudsonsbay Rideau Centre 73 Rideau St <U+0001F5D3><U+FE0F> July 3 9:45-4:30pm <U+0001F947>1st…,[ON] <U+0001F489> OTTAWA POPUP <U+2B50><U+FE0F> OPH priority neighbourhoods <U+00

In [ ]:
twitter_info <- twitter_info |>
select(created, hashtags)|>
group_by(created, hashtags)|>
summarise(freq = n())|>
ungroup()

twitter_info

In [ ]:
cleaned_hashtags_2 <- cleaned_hashtags |>
group_by(hashtags)|>
summarize(count = n())|>
arrange(desc(count))

write_csv(cleaned_hashtags_2, "cleaned_hashtags.csv")

In [ ]:
colnames(twitter_data)

twitter_data_2 <- twitter_data |>
select("place", "location", "name", "text")
twitter_data_2